# **1555. Bank Account Summary**
``` console
Table: Users

+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| user_id      | int     |
| user_name    | varchar |
| credit       | int     |
+--------------+---------+
user_id is the primary key (column with unique values) for this table.
Each row of this table contains the current credit information for each user.
 

Table: Transactions

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| trans_id      | int     |
| paid_by       | int     |
| paid_to       | int     |
| amount        | int     |
| transacted_on | date    |
+---------------+---------+
trans_id is the primary key (column with unique values) for this table.
Each row of this table contains information about the transaction in the bank.
User with id (paid_by) transfer money to user with id (paid_to).
 

Leetcode Bank (LCB) helps its coders in making virtual payments. Our bank records all transactions in the table Transaction, we want to find out the current balance of all users and check whether they have breached their credit limit (If their current credit is less than 0).

Write a solution to report.

user_id,
user_name,
credit, current balance after performing transactions, and
credit_limit_breached, check credit_limit ("Yes" or "No")
Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Users table:
+------------+--------------+-------------+
| user_id    | user_name    | credit      |
+------------+--------------+-------------+
| 1          | Moustafa     | 100         |
| 2          | Jonathan     | 200         |
| 3          | Winston      | 10000       |
| 4          | Luis         | 800         | 
+------------+--------------+-------------+

Transactions table:
+------------+------------+------------+----------+---------------+
| trans_id   | paid_by    | paid_to    | amount   | transacted_on |
+------------+------------+------------+----------+---------------+
| 1          | 1          | 3          | 400      | 2020-08-01    |
| 2          | 3          | 2          | 500      | 2020-08-02    |
| 3          | 2          | 1          | 200      | 2020-08-03    |
+------------+------------+------------+----------+---------------+

Output: 
+------------+------------+------------+-----------------------+
| user_id    | user_name  | credit     | credit_limit_breached |
+------------+------------+------------+-----------------------+
| 1          | Moustafa   | -100       | Yes                   | 
| 2          | Jonathan   | 500        | No                    |
| 3          | Winston    | 9900       | No                    |
| 4          | Luis       | 800        | No                    |
+------------+------------+------------+-----------------------+

Explanation: 
Moustafa paid $400 on "2020-08-01" and received $200 on "2020-08-03", credit (100 -400 +200) = -$100
Jonathan received $500 on "2020-08-02" and paid $200 on "2020-08-08", credit (200 +500 -200) = $500
Winston received $400 on "2020-08-01" and paid $500 on "2020-08-03", credit (10000 +400 -500) = $9990
Luis did not received any transfer, credit = $800
```

**Solution-1:**

``` sql
# Write your MySQL query statement below

SELECT u.user_id, user_name,
(credit + COALESCE(t1.amount,0)) as credit,
CASE WHEN (credit + COALESCE(t1.amount,0)) > 0 THEN 'No'
ELSE 'Yes' END AS credit_limit_breached
FROM Users u
LEFT JOIN (
SELECT user_id, SUM(amount) AS amount
FROM
(SELECT paid_by as user_id, (-amount) as amount FROM Transactions
UNION ALL
SELECT paid_to as user_id, (amount) as amount FROM Transactions)a
GROUP BY 1) t1
ON u.user_id = t1.user_id

**Solution-2:**

``` sql
SELECT
    user_id,
    user_name,
    SUM(change_in_bal) + credit AS credit,
    IF(SUM(change_in_bal) + credit < 0, "Yes", "No") AS credit_limit_breached
FROM (
SELECT
    U.user_id,
    U.user_name,
    U.credit,
    CASE WHEN U.user_id = T.paid_by THEN -T.amount
         WHEN U.user_id = T.paid_to THEN +T.amount
         ELSE 0
         END
    AS change_in_bal
FROM Users U
LEFT JOIN Transactions T
ON (U.user_id = T.paid_by OR U.user_id = T.paid_to)
      ) T
GROUP BY user_id;

**Solution-3:**

``` sql
# Write your MySQL query statement below
SELECT user_id,user_name,
IFNULL(SUM(CASE WHEN a.user_id=b.paid_by THEN -amount ELSE amount END),0)+a.credit as credit,
CASE WHEN IFNULL(SUM(CASE WHEN a.user_id=b.paid_by THEN -amount ELSE amount END),0)>=-a.credit THEN "No" ELSE "Yes" END as credit_limit_breached
FROM Users as a
LEFT JOIN Transactions as b
ON a.user_id=b.paid_by OR a.user_id=b.paid_to
GROUP BY a.user_id;

**Solution-4:**

``` sql
select user_id, user_name, 
sum(ifnull(t2.amount,0))-sum(ifnull(t1.amount,0))+credit as credit,
case when sum(ifnull(t2.amount,0))-sum(ifnull(t1.amount,0))+credit < 0 then "Yes" else "No" end as credit_limit_breached
from users u 
left join transactions t1 on u.user_id = t1.paid_by
left join transactions t2 on u.user_id = t2.paid_to
group by user_id